In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
# Hyperparameters Results

learning_rateFound=0.01
n_estimatorsFound=110
max_depthFound=8

#{'max_depth': 39, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 91}

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomOverSampling(x_train0, y_train0,5)

# Hyperparameters tuning

In [7]:
#%%script false

from scipy.stats import randint
from sklearn.ensemble import GradientBoostingClassifier
 
modelClf = GradientBoostingClassifier(random_state=42)
dic_param={
    'learning_rate': [0.01], 
    'n_estimators': randint(90,140),
    'max_depth':randint(8,14)
}

res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train, iter=2)
print(res)

KeyboardInterrupt: 

In [ ]:
%%script false
from sklearn.ensemble import GradientBoostingClassifier
 
modelClf = GradientBoostingClassifier(random_state=42)

dic_param={
    'learning_rate': [0.01], 
    'n_estimators': [125,130,135],
    'max_depth': [11,12,13]
}
res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)

In [ ]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

 

    
modelClf = GradientBoostingClassifier(random_state=42)
parameters={'learning_rate': learning_rateFound, 'n_estimators':n_estimatorsFound, 'max_depth':max_depthFound}
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)

predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
threshold=0.45

for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    y_pred_prob = modelClf.predict_proba(dfTrx[predictors])[:,1]
    y_pred_adjusted = (y_pred_prob>threshold).astype(int)

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], y_pred_adjusted,'All', True)
    
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], y_pred_adjusted)

    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)



resultMd.update_performance_nextdays_result('07-sklearn.ensemble.b-boosting','GradientBoostingClassifier','5-After oversampling', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

